# Giriş işlemleri

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import torch
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
veri_kumesi_yolu = os.path.join(kok_dizin, "sinama_sorulari.csv")
sonuc_dizini     = os.path.join(kok_dizin, "sonuclar")
# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

model_yollari = {
    "gpt2_medium_gpt4o"   : gpt2_medium_kaydetme_dizini_gpt4o,
    "gpt2_large_gpt4o"    : gpt2_large_kaydetme_dizini_gpt4o,
    "gpt2_medium_deepseek": gpt2_medium_kaydetme_dizini_deepseek,
    "gpt2_large_deepseek" : gpt2_large_kaydetme_dizini_deepseek,
}

soru_sutunu = "soru"

# Özel token tanımlamaları (sınama biçimi için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"

# Fonksiyonlar

## Soruyu girdi biçimine çevirme fonksiyonu

In [ ]:
def soruyu_girdi_bicimine_cevir(soru: str) -> str:
    """
    <SORU> ... </SORU> <CEVAP> biçiminde prompt döndürür.
    Model cevabı üretirken </CEVAP> veya EOS verdiği anda durduracağız.
    """
    soru = soru.strip()
    return f"{soru_baslangic} {soru} {soru_bitis} {cevap_baslangic} "

## Modeli yükleme fonksiyonu

In [ ]:
def model_yukle(model_yolu: str, dtype: str = "auto"):
    """
    bitsandbytes kullanmadan doğrudan AutoModelForCausalLM yükler.
    dtype:
        • "auto"  → GPU varsa float16 / bf16, yoksa float32
        • "fp16"  → torch.float16
        • "bf16"  → torch.bfloat16
        • "fp32"  → torch.float32
    """
    if dtype == "auto":
        if torch.cuda.is_available():
            major = torch.cuda.get_device_capability(0)[0]
            kullanicidtype = torch.bfloat16 if major >= 8 else torch.float16
        else:
            kullanicidtype = torch.float32
    elif dtype == "fp16": kullanicidtype = torch.float16
    elif dtype == "bf16": kullanicidtype = torch.bfloat16
    else:                 kullanicidtype = torch.float32

    print(f"\n[MODEL] {model_yolu} yükleniyor ({str(kullanicidtype)})...")
    model = AutoModelForCausalLM.from_pretrained(
        model_yolu,
        torch_dtype=kullanicidtype,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_yolu, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

## Modelden cevabı alma fonksiyonu

In [ ]:
def cevap_uret(
        model,
        tokenizer,
        prompt: str,
        *,
        max_new_tokens: int      = 1024,
        no_repeat_ngram_size: int = 3,
        repetition_penalty: float = 1.05,
        eos_token_id: int        = None):
    """
    • do_sample=False  → greedy
    • temperature/top_p dikkate alınmaz
    • no_repeat_ngram_size 3 → aynı 3-gram tekrar edemez
    • repetition_penalty >1  → tekrar eden tokenu cezalandır
    """
    if eos_token_id is None:
        eos_token_id = tokenizer.convert_tokens_to_ids("</CEVAP>")

    giris_ids = tokenizer(prompt, return_tensors="pt").to(model.device)

    cikti_ids = model.generate(
        **giris_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,                 # <-- deterministik
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
    )

    yanit = tokenizer.decode(cikti_ids[0][giris_ids["input_ids"].shape[-1]:],
                             skip_special_tokens=True)
    return yanit.split("</CEVAP>")[0].strip()

# Sonuç üretme kısmı

## Veri kümesi oku

In [ ]:
df_sorular = pd.read_csv(veri_kumesi_yolu)
print(f"{len(df_sorular)} soru yüklendi.")

## Her model için sonuç üret ve kaydet

In [ ]:
# sonuçlar dizinini oluştur
os.makedirs(sonuc_dizini, exist_ok=True)

In [ ]:
for model_etiket, model_folder in model_yollari.items():
    cikti_yolu = os.path.join(sonuc_dizini, f"{model_etiket}.csv")
    if os.path.exists(cikti_yolu):
        print(f"[SKIP] '{cikti_yolu}' zaten var, geçiliyor.")
        continue

    # model + tokenizer
    model, tokenizer = model_yukle(model_folder)
    model.eval()

    cevaplar = []
    for soru in tqdm(df_sorular["soru"], desc=model_etiket):
        prompt = soruyu_girdi_bicimine_cevir(soru)
        cevap  = cevap_uret(model, tokenizer, prompt)
        cevaplar.append(cevap)

    # sonuç DataFrame’i
    df_sonuclar = df_sorular.copy()
    df_sonuclar["cevap"] = cevaplar
    df_sonuclar.to_csv(cikti_yolu, index=False, encoding="utf-8")
    print(f"[OK] {model_etiket} sonuçları '{cikti_yolu}' dosyasına kaydedildi.")

    # belleği temizle
    del model
    torch.cuda.empty_cache()